In [18]:
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

def gap_statistic(X, max_k=10, B=10):
    """
    Computes the gap statistic for Gaussian Mixture Models clustering of X.

    Parameters
    ----------
    X : numpy array of shape (n_samples, n_features)
        The data to cluster.
    max_k : int, optional (default=10)
        The maximum number of clusters to consider.
    B : int, optional (default=10)
        The number of Monte Carlo samples to use for computing the reference distribution.

    Returns
    -------
    gap : numpy array of shape (max_k-1,)
        The gap statistic for each value of k from 1 to max_k-1.
    """
    # Compute the observed within-cluster dispersion for each k
    Wks = np.zeros(max_k-1)
    for k in range(1, max_k):
        model = GaussianMixture(n_components=k, covariance_type='full', random_state=42)
        model.fit(X)
        Wks[k-1] = model.score(X)

    # Compute the reference distribution for each k
    Bks = np.zeros(max_k-1)
    for b in range(B):
        # Generate a reference dataset
        Xref = np.random.rand(*X.shape)

        # Compute the within-cluster dispersion for each k on the reference dataset
        Wkbs = np.zeros(max_k-1)
        for k in range(1, max_k):
            model = GaussianMixture(n_components=k, covariance_type='full', random_state=42)
            model.fit(Xref)
            Wkbs[k-1] = model.score(Xref)

        Bks += Wkbs
    Bks /= B

    # Compute the standard error for each k
    s_k = np.sqrt((1 + 1/B) * np.var(Bks))

    # Compute the gap statistic for each k
    gap = np.zeros(max_k-1)
    for k in range(1, max_k):
        gap[k-1] = Bks[k-1] - Wks[k-1] + s_k * np.sqrt(1 + 1/B)

    return gap


In [ ]:
from sklearn.datasets import make_blobs

# Generate some synthetic data
X, _ = make_blobs(n_samples=300, centers=3, cluster_std=1.0, random_state=42)

# Compute the gap statistic for Gaussian Mixture Models clustering
gap = gap_statistic(X, max_k=10, B=10)

# Find the optimal number of clusters
k = np.argmin(gap) + 1

# Print the optimal number of clusters
print(f"The optimal number of clusters is {k}")


# Plot the gap statistic
plt.figure()
plt.plot(range(1, 10), gap)
plt.xlabel('Number of Clusters')
plt.ylabel('Gap Statistic')
plt.title('Gap Statistic vs. Number of Clusters')
plt.show()


# Fit a Gaussian mixture model
gmm = GaussianMixture(n_components=k, random_state=42)
gmm.fit(X)

# Predict the cluster labels
y_pred = gmm.predict(X)

# Plot the data with different colors for each cluster
colors = ['r', 'g', 'b', 'y']
plt.figure(figsize=(10, 8))
for i in range(k):
    plt.scatter(X[y_pred == i, 0], X[y_pred == i, 1], c=colors[i], label=f'Cluster {i}')
plt.legend()
plt.show()

# Print the mean and covariance matrices of each cluster
for i in range(k):
    print(f'Mean of Cluster {i}: {gmm.means_[i]}')
    print(f'Covariance Matrix of Cluster {i}:\n{gmm.covariances_[i]}\n')
